In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# import pytest
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
import requests
from bs4 import BeautifulSoup, Tag
import time
import lxml
import html5lib
import pandas as pd

In [54]:
def crawlByDepId(depId):
    options = Options()
    options.add_argument("--disable-notifications")
    # options.add_argument('--headless')
    driver = webdriver.Chrome(options = options)

    # driver.get("https://nol.ntu.edu.tw/nol/guest/index.php") # 更改網址以前往不同網頁
    # driver.get("https://nol.ntu.edu.tw/nol/coursesearch/index.php?lang=CH")
    driver.get("https://nol.ntu.edu.tw/nol/coursesearch/search_for_02_dpt.php")
    select_element = driver.find_element(By.ID, 'dptname')
    select = Select(select_element)
    select.select_by_value(depId)
    numInPage = driver.find_element(By.ID, "page_cnt")
    numInPage.clear()
    numInPage.send_keys("150")
    enterButton = driver.find_element(By.NAME, "Submit22")
    enterButton.click()

    tables = driver.find_elements(By.TAG_NAME, "table")
    tablePreferable = []
    for i, t in enumerate(tables):
        print(i)
        try:
            if t.get_attribute("bordercolorlight") == "#CCCCCC" and t.get_attribute("bordercolordark") == "#CCCCCC":
                tablePreferable.append(t)
        except:
            print("Not this one")

    if len(tablePreferable) != 1:
        print("Errorrrr")
    else:
        table = tablePreferable[0]

        classIdList = []
        classNameList = []
        classSummaryList = []
        rows = table.find_elements(By.TAG_NAME, "tr")
        for i2, row in enumerate(rows[1:]):
            print("Dealing with class", f"{i2}/{len(rows)}")

            # 获取行中的所有单元格
            cells = row.find_elements(By.TAG_NAME, "td")
            
            classIdList.append(cells[0].text)
            classElement = cells[4]
            classNameList.append(classElement.text)
            print(classElement.text)
            href = classElement.find_element(By.TAG_NAME, "a").get_attribute("href")

            response = requests.get(href)
            # response.encoding = response.apparent_encoding
            if response.status_code == 200:
                # 使用 BeautifulSoup 解析 HTML
                soup = BeautifulSoup(response.content, 'html5lib')
                if soup.find('strong', string = '為確保您我的權利,請尊重智慧財產權及不得非法影印'):
                    start = soup.find('div', string='課程概述')
                    end = soup.find('div', string='預期每週課後學習時數').previous_element
                    result = ''
                    current_element = start.next_element
                    while current_element and current_element != end:
                        if isinstance(current_element, Tag) and current_element.name == 'td':
                            result += str(current_element.get_text()) + " "
                        current_element = current_element.next_element

                    result = result.replace("\n", " ")
                    result = " ".join(result.split())
                    print(result)
                else:
                    result = ""
                
                classSummaryList.append(result)

            # driver2 = webdriver.Chrome(options = options)
            # driver2.get(href)
            # ancher = driver.find_element(By.PARTIAL_LINK_TEXT, "課程大綱")
            # print(ancher.text)

            


            # classElement.click()

            # original_window = driver.current_window_handle
            # wait = WebDriverWait(driver, 10)
            # wait.until(lambda d: len(d.window_handles) > 1)
            # new_window = [window for window in driver.window_handles if window != original_window][0]
            # driver.switch_to.window(new_window)

            # # driver.close()
            # driver.switch_to.window(original_window)
            # enterButton.get_attribute('value')
            
        data = {'ClassId': classIdList, 'ClassName': classNameList, 'classSummary':classSummaryList}  
        df = pd.DataFrame(data)  
        df.to_csv(f"{depId}.csv", encoding="utf-8-sig")



    
    # table = driver.find_element_by_id("table_id")







In [55]:
for depId in ["7250", "7050"]:
    crawlByDepId(depId)

0
1
2
3
4
5
6
7
8
Dealing with class 0/37
 博士論文
Dealing with class 1/37
 高等資訊管理
In this course, we will cover several research methods widely employed in IS research (e.g., survey, experiment, design science, etc). In addition, being a multidisciplinary field of study, IS research (especially behavioral IS research) draws upon various existing theories developed in other disciplines to explain or predict a phenomenon of interest to IS field (e.g., individuals’ or organizations’ behavior in a socio-technical context). Hence, we will also discuss theories that are commonly adopted by IS research. Besides the lectures by the instructor, the course will also be conducted in the paper-presentation-and-critic format. Through reading and presenting assigned papers, students are expected to gain an understanding on contemporary IS research and to establish appropriate perception about what a research is. Complementarily, through paper discussions and critics, students are expected to improve t

In [43]:
response = requests.get("https://nol.ntu.edu.tw/nol/coursesearch/print_table.php?course_id=725%20U3960&class=&dpt_code=7250&ser_no=51682&semester=112-1&lang=CH")
# response.encoding = response.apparent_encoding
if response.status_code == 200:
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(response.content, 'html5lib')
    if soup.find('strong', string = '為確保您我的權利,請尊重智慧財產權及不得非法影印'):
        start = soup.find('div', string='課程概述')
        end = soup.find('div', string='預期每週課後學習時數').previous_element
        result = ''
        current_element = start.next_element
        while current_element and current_element != end:
            if isinstance(current_element, Tag) and current_element.name == 'td':
                result += str(current_element.get_text()) + " "
            current_element = current_element.next_element

        result = result.replace("\n", " ")
        result = " ".join(result.split())
        print(result)


2023/9/1 The details of this course is announced: https://docs.google.com/document/d/1jtMt4F1XYIiEOAWxZ2aWwzFpz4AZDlkxcRQ80iVhtOo/edit?usp=sharing 2023/8/21 1. Again, there are no authorization code of this course since it is controlled by the system. 2. If you want to audit / sit in on this course, please come to the classroom in the first week. Please do not send the email to the teacher or TAs. 2023/8/7 1. The details of this course will be updated on the beginning of Sept and it will be very different to the that in the last year. 2. There are no authorization code because it is controlled by the system and the limit of the number of student is 80. Therefore, if you ask the authorization code, I won't reply you. ---------------------------------------------------------------------------------- 本課程內容涵蓋網頁設計、後端資料庫和網頁轉化成手機APP等基本概念及其運用，例如：HTML5、CSS3、JavaScript、jQuery、jQuery Mobile、SQL、NoSQL以及Cordova等。 在整個課程中，藉由以某領域為主題搭配專案管理之手段，以解決真實問題為目標之專案製作，進一步達到學習之目的，提升同學學習成效。藉由分組討論與專案報告讓同學們分享解決問題的過程

"課程概述 2023/9/1 The details of this course is announced: https://docs.google.com/document/d/1jtMt4F1XYIiEOAWxZ2aWwzFpz4AZDlkxcRQ80iVhtOo/edit?usp=sharing 2023/8/21 1. Again, there are no authorization code of this course since it is controlled by the system. 2. If you want to audit / sit in on this course, please come to the classroom in the first week. Please do not send the email to the teacher or TAs. 2023/8/7 1. The details of this course will be updated on the beginning of Sept and it will be very different to the that in the last year. 2. There are no authorization code because it is controlled by the system and the limit of the number of student is 80. Therefore, if you ask the authorization code, I won't reply you. ---------------------------------------------------------------------------------- 本課程內容涵蓋網頁設計、後端資料庫和網頁轉化成手機APP等基本概念及其運用，例如：HTML5、CSS3、JavaScript、jQuery、jQuery Mobile、SQL、NoSQL以及Cordova等。 在整個課程中，藉由以某領域為主題搭配專案管理之手段，以解決真實問題為目標之專案製作，進一步達到學習之目的，提升同學學習成效。藉由分組討論與專案報告讓同學們分享解

In [6]:
# 以Google首頁為例
# 定位搜尋框
element = driver.find_element("class name", "gLFyf.gsfi")
# 傳入字串
element.send_keys("Selenium Python")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".gLFyf.gsfi"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7C4E182B2+55298]
	(No symbol) [0x00007FF7C4D85E02]
	(No symbol) [0x00007FF7C4C405AB]
	(No symbol) [0x00007FF7C4C8175C]
	(No symbol) [0x00007FF7C4C818DC]
	(No symbol) [0x00007FF7C4CBCBC7]
	(No symbol) [0x00007FF7C4CA20EF]
	(No symbol) [0x00007FF7C4CBAAA4]
	(No symbol) [0x00007FF7C4CA1E83]
	(No symbol) [0x00007FF7C4C7670A]
	(No symbol) [0x00007FF7C4C77964]
	GetHandleVerifier [0x00007FF7C5190AAB+3694587]
	GetHandleVerifier [0x00007FF7C51E728E+4048862]
	GetHandleVerifier [0x00007FF7C51DF173+4015811]
	GetHandleVerifier [0x00007FF7C4EB47D6+695590]
	(No symbol) [0x00007FF7C4D90CE8]
	(No symbol) [0x00007FF7C4D8CF34]
	(No symbol) [0x00007FF7C4D8D0

In [ ]:
driver.close() # 關閉瀏覽器視窗